In [66]:
import numpy as np
from lr_utils import load_dataset
import h5py
import time

In [2]:
def sigmoid(z):
    a = 1/ (1+ np.exp(-z))
    return a

In [3]:
def RELU(z):
    a = np.maximum(0, z)
    return a

In [4]:
def tanh(z):
    a = np.tanh(z)
    return a

In [75]:
def initialize_zero_W(size_of_w):
    w = np.zeros([size_of_w, 1])
    return w
def initialize_zero_b(size_of_b):
    b = np.zeros([size_of_b])
    
    return b

In [80]:
def initialize_random(size_of_w):
    w = np.random.rand(size_of_w,1)
    print(w.shape)
    return w

In [13]:
def forward_propagation(W, X, b):
    z = sigmoid(np.dot(W.T, X) + b)
    return z

In [45]:
def backward_propagation(A, Y, X):
    m = Y.shape[1]
    dw = 1 / m * np.dot(X, (A - Y).T)
    db = 1 / m * np.sum(A - Y)
    return dw, db
def calculate_cost(Y, A):
    m = Y.shape[1]
    cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    return cost

In [39]:
def perdiction(Z):
    for i in range(Z.shape[1]):
        if Z[0][i] > 0.5:
            Z[0][i] = 1
        else:
            Z[0][i] = 0
    return Z
        

In [106]:
def model(X, Y, learning_rate = 0.01):
    W = initialize_zero_W(X.shape[0])
    b = initialize_zero_b(1)
    for i in range(40000):
        A = forward_propagation(W, X, b)
        #print(A)
        dw, db = backward_propagation(A, Y, X)
        W -= learning_rate * dw
        b -= learning_rate * db
        if i % 1000 == 0:
            print(calculate_cost(Y, A))
    count = 0
    Z = forward_propagation(W, X, b)
    for i in range(Y.shape[1]):    
        Z = perdiction(Z)
        if Z[0][i] == Y[0][i]:
            count += 1
    print(count/Y.shape[1] * 100)
    return W, b

In [104]:
def test_deep(Y, X, W, b):
    count = 0
    Z = forward_propagation(W, X, b)
    for i in range(Y.shape[1]):
        Z = perdiction(Z)
        if Z[0][i] == Y[0][i]:
            count += 1
    print(count/Y.shape[1] * 100)

In [107]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
#print(train_set_x_orig.shape, train_set_y.shape, test_set_x_orig.shape, test_set_y.shape)
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

W, b = model(train_set_x, train_set_y)
test_deep(test_set_y, test_set_x, W, b)

0.6931471805599453
0.11647833125783899
0.0698603560502306
0.049484807864292366
0.038111220065750795
0.030898126764563588
0.025935721991913505
0.022322251603728974
0.019578306706133107
0.017426330533589742
0.015694895547028916
0.014272594950365437
0.013083985844922543
0.012076212679338645
0.011211171633234089
0.010460716682854646
0.009803616348075887
0.009223559524177565
0.008707812219409083
0.008246290581776157
0.007830907364920918
0.007455102303620898
0.007113498825660645
0.006801649214592465
0.006515842771072725
0.006252959549429186
0.006010357536986887
0.005785784695102938
0.005577309705262949
0.00538326694447121
0.005202212396315293
0.005032888046534279
0.004874192919685392
0.004725159356986327
0.004584933462473983
0.004452758888233058
0.0043279633125988095
0.004209947104140405
0.00409817377044463
0.003992161872567563
100.0
72.0
